# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

I) Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.

II) Se debe aplicar alguna técnica de feature engineering para mejorar la data de entrada a los modelos. Si no es posible o útil, fundamentar el motivo por el cual no se realizará.

III) Por cada modelo, se debe entrenarlo y evaluar su comportamiento con la métrica elegida. Sacar conclusiones respecto a los motivos por los cuales el modelo funciona bien o mal.

IV) Se deben utilizar técnicas que garanticen que los modelos no están sobreentrenando sin que nos demos cuenta.

V) Determinar el valor final de la métrica que podría ser informado al cliente, utilizando técnicas que permitan obtener un valor lo más realista posible. Fundamentar.